In [1]:
import os
import numpy as np
import pandas as pd
import pandapower as pp
from pandapower.control import ConstControl
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries

In [26]:
TIMESTEPS = 35040 # 365 days with 15min resolution
V_REF = 1.0 # pu
V_NOM = 416 # V

CURRENT_SCENARIO = 'initial'

working_folder = os.getcwd()
root_dir = os.path.abspath(os.path.join(working_folder, os.pardir))
network_path = os.path.join(root_dir, 'data', CURRENT_SCENARIO,'network')
data_path = os.path.join(root_dir, 'data', CURRENT_SCENARIO)

# load network
net = pp.from_json(os.path.join(network_path,'IEEE_modified_LV_Feeder.json'))
# load required dataframes
# aggregate node consumption/generation data
aggregate_node_data = pd.read_csv(os.path.join(data_path,"aggregate_node_info.csv"))
generators_df = pd.read_csv(os.path.join(data_path,"pv_data","pv_info.csv"))


## Data source

Data taken from csv data provided in IEEE benchmark grid

In [27]:
def create_load_data_source(aggregate_df, data_path):
    profiles = pd.DataFrame()
    for i, row in aggregate_df.iterrows():
        # add yealy load profile
        load_file_path = os.path.join(data_path,"load_data","15min",row['load_profile_name'])
        load_profile = pd.read_csv(load_file_path).consumption_kW.values * 1e-3  # read in mw

        # add yearly ev profile
        ev_profile = [0] * len(load_profile)
        if pd.notna(row['ev_profile_name']):
            ev_file_path = os.path.join(data_path,"ev_data","15min",row['ev_profile_name'])
            ev_profile = pd.read_csv(ev_file_path).charging_power_kW.values * 1e-3  # read in mw 

        profiles[row['load_name'] + '_P'] = np.add(load_profile, ev_profile)
        
        # add reactive power column
        profiles[row['load_name'] + '_Q'] = 0 

    ds = DFData(profiles)
    return profiles, ds

def create_gen_data_source(gens_df, data_path):
# carefull data is in watt
    profiles = pd.DataFrame()    
    for id, filepath in (gens_df[['id','profile_name']].values):        
        profiles['PV' + str(id) + '_P'] = pd.read_csv(os.path.join(data_path,"pv_data","15min",filepath)).p_w.values * 1e-6  # read in mw
        
    profiles['PV_Q'] = 0

    ds = DFData(profiles)
    return profiles, ds

## Time series controller

P, Q values entered using P and power factor(cos_phi) data

In [28]:
def create_load_controllers(net, aggregate_df, ds):
    ConstControl(net, element='load', variable='p_mw', element_index=aggregate_df.index,
                 data_source=ds, profile_name=aggregate_df.load_name+'_P')
    ConstControl(net, element='load', variable='q_mvar', element_index=aggregate_df.index,
                 data_source=ds, profile_name=aggregate_df.load_name+'_Q')
    return net

def create_gen_controllers(net, gens_df, ds):
    ConstControl(net, element='sgen', variable='p_mw', element_index=gens_df.index,
                 data_source=ds, profile_name='PV'+gens_df['id'].astype(str)+'_P')
    ConstControl(net, element='sgen', variable='q_mvar', element_index=gens_df.index,
                 data_source=ds, profile_name='PV_Q')
    return net

## Format output

- Bus voltage magnitudes and angles for each time step
- Aggregated Sum of P & Q values for each time step

In [29]:
def create_output_writer(net, time_steps, output_dir):
    ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".json")
    ow.log_variable('res_trafo', 'p_lv_mw', index=net.trafo.index)
    ow.log_variable('res_trafo', 'q_lv_mvar', index=net.trafo.index)
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_bus', 'va_degree')
    ow.log_variable('res_line', 'loading_percent')
    return ow

In [30]:
def timeseries_calculation(net, n_timesteps, output_dir, aggregate_df, gens_df, data_path):
    # create data source
    load_profiles, ds_load = create_load_data_source(aggregate_df, data_path)
    sgen_profiles, ds_sgen = create_gen_data_source(gens_df, data_path)

    # create controllers (to control P values of the load and the sgen)
    create_load_controllers(net, aggregate_df, ds_load)
    create_gen_controllers(net, gens_df, ds_sgen)

    # time steps to be calculated. Could also be a list with non-consecutive time steps
    time_steps = range(0, n_timesteps)

    # 4. the output writer with the desired results to be stored to files.
    ow = create_output_writer(net, time_steps, output_dir=output_dir)

    # 5. the main time series function
    run_timeseries(net, time_steps)

# Main Function

In [32]:
output_dir = os.path.join(data_path, "time_series_calculation")
print("Results can be found in data folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_calculation(net, TIMESTEPS, output_dir, aggregate_node_data, generators_df, data_path)

Results can be found in data folder: c:\Users\dgont\OneDrive\Documents\VScode\GridOptiPlan\data\initial\time_series_calculation


C:\Users\dgont\AppData\Local\Temp\ipykernel_31864\2917488755.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  profiles[row['load_name'] + '_P'] = np.add(load_profile, ev_profile)
C:\Users\dgont\AppData\Local\Temp\ipykernel_31864\2917488755.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  profiles[row['load_name'] + '_Q'] = 0
C:\Users\dgont\AppData\Local\Temp\ipykernel_31864\2917488755.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo